# Import findspark and initialize. 
import findspark
findspark.init()

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Analyzing Home Sales") \
    .getOrCreate()

# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [2]:
home_df = spark.read\
               .format("csv")\
               .option("header", "true")\
               .load("Resources/home_sales_revised.csv")
home_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_df.show()

In [3]:
# 2. Create a temporary view of the DataFrame.

home_df.createOrReplaceTempView('home_sales')

In [4]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query1 = """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(query1).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [5]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query2 =  """
SELECT
  YEAR(date_built) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY YEAR(date_built)
ORDER BY YEAR DESC
"""
spark.sql(query2).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2017|    292676.79|
|2016|    290555.07|
|2015|     288770.3|
|2014|    290852.27|
|2013|    295962.27|
|2012|    293683.19|
|2011|    291117.47|
|2010|    292859.62|
+----+-------------+



In [6]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query3 = """
SELECT
  YEAR(date_built) AS YEAR_BUILT,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
and sqft_living >= 2000
and floors = 2
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT DESC
"""
spark.sql(query3).show()


+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [7]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
import time
start_time = time.time()

query4 = """
SELECT
  view, 
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(query4).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 2.3087377548217773 seconds ---


In [8]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [9]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [10]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql(query4).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.5741782188415527 seconds ---


In [11]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

Choose Pre built for Apache Hadoop 2.7 to work the below code

In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

#home_df.write.parquet('home_parquet', mode='overwrite')
#home_df.write.partitionBy('date_built').parquet("parquet/p_home_sales",mode="overwrite")

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
home_df.write.mode("overwrite").partitionBy("date_built").parquet("parquet_home_sales")

In [13]:
# 11. Read the formatted parquet data.
parquet_home_df = spark.read.parquet('parquet_home_sales')

In [14]:
# 12. Create a temporary table for the parquet data.
parquet_home_df.createOrReplaceTempView('parquet_home_sales')

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
query5 = """
SELECT
  view, 
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM parquet_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(query4).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.6546757221221924 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False